# Werewolf Game

This is a demo of how to use AgentScope to play the Werewolf game, where six
 agents play the roles of werewolves and villagers. The details can be found
  in `examples/werewolf/README.md`.

To install AgentScope, please follow the steps in [README.md](../README.md#installation).

First, we need to set the model configs for the agents.

In [1]:
model_configs = [
    {
        "type": "openai",
        "name": "gpt-3.5-turbo",
        "api_key": "xxx",
        "organization": "xxx",
        "generate_args": {
            "temperature": 0.5
        }
    },
    {
        "type": "post_api",
        "name": "my_post_api",
        "api_url": "https://xxx",
        "headers": {
        },
        "json_args": {
        }
    }
]

The configs for each agent are as follows. There are six agents, each playing a role in the game. Here, `DictDialogAgent` is a specific class in AgentScope. The details can be found in `src/agentscope/agents/dict_dialog_agent.py`. The details about `sys_prompt` for each player can be found in `examples/werewolf/configs/agent_configs`.

In [8]:
agent_configs = [
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player1",
            "sys_prompt": "Act as a player in a werewolf game. You are Player1 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player2",
            "sys_prompt": "Act as a player in a werewolf game. You are Player2 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player3",
            "sys_prompt": "Act as a player in a werewolf game. You are Player3 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player4",
            "sys_prompt": "Act as a player in a werewolf game. You are Player4 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player5",
            "sys_prompt": "Act as a player in a werewolf game. You are Player5 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player6",
            "sys_prompt": "Act as a player in a werewolf game. You are Player6 ...",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    }
]

We also need to set the prompts for different players in differents states. The details can be found in `examples/werewolf/prompt.py`.

In [9]:
"""Used to record prompts, will be replaced by configuration"""


class Prompts:
    """Prompts for werewolf game"""

    to_wolves = (
        "{}, you are werewolves. ..."
    )

    to_wolves_vote = (
        "Which player do you vote to kill? ..."
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're witch. ..."
    )

    to_witch_poison = (
        "Would you like to eliminate one player? ..."
    )

    to_seer = (
        "{}, you're seer. ..."
    )

    to_seer_result = "Okay, the role of {} is {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. ..."
    )

    to_all_vote = (
        "Now the alive players are {}. ..."
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

Now we can turn our attention to the main function of the game. Run the following codes and enjoy the game.

In [ ]:
"""A werewolf game implemented by agentscope."""
from functools import partial
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.pipelines.functional import sequentialpipeline
import agentscope

import sys
sys.path.append('../examples/werewolf/')
from prompt import Prompts
from utils import (
    check_winning,
    update_alive_players,
    majority_vote,
    extract_name_and_id,
    n2s,
)


# default settings
HostMsg = partial(Msg, name="Moderator", echo=True)
healing, poison = True, True
MAX_WEREWOLF_DISCUSSION_ROUND = 3
MAX_GAME_ROUND = 6
# read model and agent configs, and initialize agents automatically
survivors = agentscope.init(
    model_configs=model_configs,
    agent_configs="../examples/werewolf/configs/agent_configs.json",
)
roles = ["werewolf", "werewolf", "villager", "villager", "seer", "witch"]
wolves, witch, seer = survivors[:2], survivors[-1], survivors[-2]

# start the game
for i in range(1, MAX_GAME_ROUND + 1):
    # night phase, werewolves discuss
    hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
    with msghub(wolves, announcement=hint) as hub:
        for _ in range(MAX_WEREWOLF_DISCUSSION_ROUND):
            x = sequentialpipeline(wolves)
            if x.get("agreement", False):
                break

        # werewolves vote
        hint = HostMsg(content=Prompts.to_wolves_vote)
        votes = [extract_name_and_id(wolf(hint).content)[0] for wolf in wolves]
        # broadcast the result to werewolves
        dead_player = [majority_vote(votes)]
        hub.broadcast(
            HostMsg(content=Prompts.to_wolves_res.format(dead_player[0])),
        )

    # witch
    healing_used_tonight = False
    if witch in survivors:
        if healing:
            hint = HostMsg(
                content=Prompts.to_witch_resurrect.format_map(
                    {"witch_name": witch.name, "dead_name": dead_player[0]},
                ),
            )
            if witch(hint).get("resurrect", False):
                healing_used_tonight = True
                dead_player.pop()
                healing = False

        if poison and not healing_used_tonight:
            x = witch(HostMsg(content=Prompts.to_witch_poison))
            if x.get("eliminate", True):
                dead_player.append(extract_name_and_id(x.content)[0])
                poison = False

    # seer
    if seer in survivors:
        hint = HostMsg(
            content=Prompts.to_seer.format(seer.name, n2s(survivors)),
        )
        x = seer(hint)

        player, idx = extract_name_and_id(x.content)
        role = "werewolf" if roles[idx] == "werewolf" else "villager"
        hint = HostMsg(content=Prompts.to_seer_result.format(player, role))
        seer.observe(hint)

    survivors, wolves = update_alive_players(survivors, wolves, dead_player)
    if check_winning(survivors, wolves, "Moderator"):
        break

    # daytime discussion
    content = (
        Prompts.to_all_danger.format(n2s(dead_player))
        if dead_player
        else Prompts.to_all_peace
    )
    hints = [
        HostMsg(content=content),
        HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors))),
    ]
    with msghub(survivors, announcement=hints) as hub:
        # discuss
        x = sequentialpipeline(survivors)

        # vote
        hint = HostMsg(content=Prompts.to_all_vote.format(n2s(survivors)))
        votes = [extract_name_and_id(_(hint).content)[0] for _ in survivors]
        vote_res = majority_vote(votes)
        # broadcast the result to all players
        result = HostMsg(content=Prompts.to_all_res.format(vote_res))
        hub.broadcast(result)

        survivors, wolves = update_alive_players(survivors, wolves, vote_res)

        if check_winning(survivors, wolves, "Moderator"):
            break

        hub.broadcast(HostMsg(content=Prompts.to_all_continue))
